__Vi bygger om logiken för att plocka ut 14 hästar när vi plockat bort de hästar som är i VNUM (1,2,3).__

In [1]:
# Common imports
import numpy as np
import os
import pandas as pd


In [2]:
# Necessary Sklearn objects used in the analysis
from sklearn.metrics import roc_curve, auc
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn import preprocessing

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Imputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion

In [3]:
# DataPiplene är en pythonfil som innehåller den egenkostruerade klassen DataFrameSelector
from DataPipeline import DataFrameSelector
from sklearn.externals import joblib

/Users/ulf/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


__Läser in filen som ska köras__

In [4]:
df1 = pd.read_excel('RealDataV75TillUffeE20200125.xlsx')

In [5]:
# Plockar bort hästara i VNUM (1,2,3) som inte ska vara med
df1 = df1[~df1.VNUM.isin([1,2,3])]    

In [6]:
df1 = df1.sort_values(['Lopp','Hast'], ascending = ['True','False']).reset_index()

In [7]:
# Importera model som ska användas
rf = joblib.load('Travmodel_v9.pkl')

In [8]:
# Import av datapipeline som ska användas
full_pipeline = joblib.load('Pipeline_v9.pkl')

__Kör modellen och plockar fram de 14 hästarna med högst score__

In [9]:
features = full_pipeline.fit_transform(df1)

In [10]:
predict = rf.predict_proba(features)

In [11]:
predict_cutoff = pd.DataFrame({'Prob0':predict[:,0],'Prob1':predict[:,1]}).sort_values('Prob1', ascending = False).iloc[14]['Prob1']

In [12]:
y_pred = np.where( predict[:,1] > predict_cutoff,1,0)

In [13]:
# Gör om scorad array till en lista - multidimensionell
stack = predict.tolist()

In [14]:
# Sedan plockar vi den scorade sannolikheten att vara vinnare - 1
# Transformerar denna sannolikhet till en dataframe
last = []
for x in stack:
    last.append(x[1])
scored = {'Score':last}
# Konverterar till Dataframe
df_scored = pd.DataFrame.from_dict(scored)

In [15]:
# Gör om till lista och stoppar in i ett dictionary
pred = y_pred.tolist()
prediction = {'Predict':pred}
# Konverterar till Dataframe
df_pred = pd.DataFrame.from_dict(prediction)

In [16]:
df_predict1 = df1.merge(df_pred, right_index = True, left_index = True)
df_predict2 = df_predict1.merge(df_scored,  right_index = True, left_index = True) 

In [17]:
df1_out = df_predict2[df_predict2.Predict == 1][['Lopp','Hast','Score']]. \
sort_values(['Lopp','Score'], ascending = [True,False])
df1_out

,Lopp,Hast,Score
0,1,2,0.134068
16,2,11,0.161082
13,2,8,0.101458
21,3,5,0.099442
28,4,3,0.095875
29,4,4,0.081932
40,5,8,0.159772
39,5,7,0.127126
38,5,6,0.119123
37,5,3,0.098421


__Export till excel__

In [18]:
df1_out.to_excel('RealOuttake20200125.xlsx', index = False)